# Исследование музыкальных предпочтений жителей Москвы и Санкт-Петербурга

С использованием данных, предоставленных музыкальным сервисом, в проекте сопоставляются предпочтения и особенности активности пользователей двух столиц - Москвы и Петербурга.

**Цель исследования** — проверить гипотезы о том, что:
1. Активность пользователей зависит от дня недели. Причём в Москве и Петербурге это проявляется по-разному.
2. В понедельник утром в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.
3. Москва и Петербург предпочитают разные жанры музыки. В Москве чаще слушают поп-музыку, в Петербурге — русский рэп.

Данные предоставлены в виде csv-файла, в котором 7 столбцов:

* `userID` — идентификатор пользователя;
* `Track` — название трека;  
* `artist` — имя исполнителя;
* `genre` — название жанра;
* `City` — город пользователя;
* `time` — время начала прослушивания;
* `Day` — день недели.

## Обзор данных

**Импортируем библиотеку pandas и прочитаем предоставленный csv-файл. Запишем его в датафрем `df`**

In [1]:
import pandas as pd

try:
    df = pd.read_csv('/datasets/yandex_music_project.csv')
except:
    df = pd.read_csv('https://code.s3.yandex.net/datasets/yandex_music_project.csv')

**Выведем на экран первые десять строк датафрейма**

In [2]:
df.head(10)

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
5,842029A1,Преданная,IMPERVTOR,rusrap,Saint-Petersburg,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Moscow,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Moscow,20:47:49,Wednesday
8,8FA1D3BE,И вновь продолжается бой,NaN,ruspop,Moscow,09:17:40,Friday
9,E772D5C0,Pessimist,NaN,dance,Saint-Petersburg,21:20:49,Wednesday


**Получим общую информацию о датафрейме**

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


В таблице семь столбцов. Тип данных во всех столбцах — `object`.
Количество значений в столбцах различается. Значит, в данных есть пропущенные значения. Кроме того, в названиях столбцов присутствуют нарушения стиля, которые могут в дальнейшем помешать работе с данными:
1. Сочетаются заглавные и строчные буквы
2. Есть лишние пробелы
3. Вместо snake_case используется camelCase

В каждой строке таблицы — данные о прослушанном треке. Часть столбцов описывает саму композицию: название, исполнителя и жанр. Остальные данные рассказывают о пользователе: из какого он города, когда он слушал музыку.

Предварительно можно утверждать, что данных достаточно для проверки гипотез. Но встречаются пропуски в данных, а в названиях колонок — расхождения с хорошим стилем.

Чтобы двигаться дальше, нужно устранить проблемы в данных.

## Предобработка данных
**Исправим стиль в заголовках столбцов, поработаем с пропусками и проверим данные на дубликаты.**

**Выведем на экран названия столбцов:**

In [4]:
df.columns

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

**Переименуем столбцы так:**
* `'  userID'` → `'user_id'`;
* `'Track'` → `'track'`;
* `'  City  '` → `'city'`;
* `'Day'` → `'day'`.
<br>

**Для этого создадим словарь, в котором ключами будут старые имена столбцов, а значениями - новые, и переименуем столбцы:**

In [5]:
new_columns = {'  userID':'user_id', 'Track':'track', '  City  ':'city', 'Day':'day'}
df = df.rename(columns = new_columns)

**Проверим результат. Для этого ещё раз выведем на экран названия столбцов:**

In [6]:
df.columns

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

**Оценим количество пропущенных значений в таблице:**

In [7]:
df.isna().sum()

user_id       0
track      1231
artist     7203
genre      1198
city          0
time          0
day           0
dtype: int64

Не все пропущенные значения влияют на исследование. Так, пропуски в `track` и `artist` не важны для исследования. Однако пропуски в `genre` могут помешать сравнить музыкальные вкусы в Москве и Санкт-Петербурге. Возможность установить причину пропусков отсутствует, поэтому сделаем следующее:
* заполним пропуски явными обозначениями;
* оценим, насколько они повредят расчётам.

**Заменим пропущенные значения в столбцах `track`, `artist` и `genre` на строку `'unknown'`. Для этого создадим список `columns_to_replace`, переберем его элементы циклом `for` и для каждого столбца выполним замену пропущенных значений:**

In [8]:
columns_to_replace = ['track', 'artist', 'genre']
for column in columns_to_replace:
    df[column] = df[column].fillna('unknown')       

**Убедимся, что в таблице не осталось пропусков. Для этого ещё раз посчитаем пропущенные значения:**

In [9]:
df.isna().sum()

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64

### Дубликаты

**Посчитаем полностью дублирующиеся строки датафрейма:**



In [10]:
df.duplicated().sum()

3826

**Удалим дубликаты строк:**

In [11]:
df = df.drop_duplicates()

**Избавимся от неявных дубликатов в колонке `genre`. Например, название одного и того же жанра может быть записано немного по-разному. Для этого отсортируем столбец датафрейма `df` в алфавитном порядке и выведем список уникальных значений в нем:**

In [12]:
genre = df['genre']
genre.sort_values().unique()

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

**В списке присутствуют неявные дубликаты названия `hiphop`:**

* *hip*,
* *hop*,
* *hip-hop*.

**Заменим их на `hiphop`:**

In [13]:
duplicates = ['hip', 'hop', 'hip-hop']
df['genre'] = df['genre'].replace(duplicates,'hiphop')

**Кроме того, в списке есть еще дубликаты жанров музыки: 'electronic' и 'электроника'. Приведем их к единообазному 'electronic':**

In [14]:
df['genre'] = df['genre'].replace('электроника','electronic')


**Еще есть неидентифицированный жанр 'ïîï'. Проверим, как часто он встречается в нашем датасете:"** 

In [15]:
df.loc[df['genre'] == 'ïîï']['user_id'].count()

1

**В датасете есть лишь одна строка с таким жанром. Удалим ее:** 

In [16]:
df = df[df['genre'] != 'ïîï']

**Проверим, успешно ли прошло удаление дубликатов в столбце `genre`:**

In [17]:
df['genre'].sort_values().unique()

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

**Выводы**

Предобработка обнаружила три проблемы в данных:

- нарушения в стиле заголовков,
- пропущенные значения,
- дубликаты — явные и неявные.

Мы исправили заголовки, чтобы упростить работу с таблицей. Пропущенные значения заменили на `'unknown'`. Ещё предстоит увидеть, не повредят ли исследованию пропуски в колонке `genre`.

Теперь можно перейти к проверке гипотез.

## Проверка гипотез

### Сравнение поведения пользователей двух столиц

Первая гипотеза утверждает, что пользователи по-разному слушают музыку в Москве и Санкт-Петербурге. Проверим это предположение по данным о трёх днях недели — понедельнике, среде и пятнице. Для этого:

* Разделим пользователей Москвы и Санкт-Петербурга.
* Сравним, сколько треков послушала каждая группа пользователей в понедельник, среду и пятницу.


**Для начала оценим активность пользователей в каждом городе:**

In [18]:
df.groupby('city')['city'].count()

city
Moscow              42740
Saint-Petersburg    18512
Name: city, dtype: int64

В Москве прослушиваний больше, чем в Петербурге. Однако, из этого не следует, что московские пользователи чаще слушают музыку. Просто самих пользователей в Москве больше.

**Теперь сгруппируем данные по дню недели и посчитайте прослушивания в понедельник, среду и пятницу (в датасете есть информация о прослушиваниях только за эти дни):**

In [19]:
# Подсчёт прослушиваний в каждый из трёх дней
df.groupby('day')['day'].count()

day
Friday       21839
Monday       21354
Wednesday    18059
Name: day, dtype: int64

В среднем пользователи из двух городов менее активны по средам. Но картина может измениться, если рассмотреть каждый город в отдельности.

**Создадим функцию `number_tracks()`, которая посчитает прослушивания для заданного дня и города:**

In [23]:
# Функция для подсчёта прослушиваний для конкретного города и дня.
# С помощью последовательной фильтрации с логической индексацией она
# сначала получит из исходной таблицы строки с нужным днём,
# затем из результата отфильтрует строки с нужным городом,
# методом count() посчитает количество значений в колонке user_id.
# Это количество функция вернёт в качестве результата

def number_tracks(day, city):
    unique_plays = df.loc[(df['day'] == day) & (df['city'] == city)]['user_id'].count()
    return unique_plays

**Вызовем функцию `number_tracks` и посчитаем количество прослушиваний в Москве по понедельникам:**

In [24]:
number_tracks('Monday', 'Moscow')

15740

**Количество прослушиваний в Санкт-Петербурге по понедельникам:**

In [25]:
number_tracks('Monday', 'Saint-Petersburg')

5614

**Количество прослушиваний в Москве по средам:**

In [26]:
number_tracks('Wednesday', 'Moscow')

11056

**Количество прослушиваний в Санкт-Петербурге по средам:**

In [27]:
number_tracks('Wednesday', 'Saint-Petersburg')

7003

**Количество прослушиваний в Москве по пятницам:**

In [28]:
number_tracks('Friday', 'Moscow')

15944

**Количество прослушиваний в Санкт-Петербурге по пятницам:**

In [29]:
number_tracks('Friday', 'Saint-Petersburg')

5895

**С помощью конструктора `pd.DataFrame` создадим таблицу, где:**
* названия колонок — `['city', 'monday', 'wednesday', 'friday']`;
* данные — результаты, которые мы получили с помощью `number_tracks`

In [30]:
column_names = ['city', 'monday', 'wednesday', 'friday']
data = [['Moscow', 15740, 11056, 15944], ['Saint_Petersburg', 5614, 7003, 5895]]

unique_plays_by_cities = pd.DataFrame(data=data, columns=column_names)
print(unique_plays_by_cities)

               city  monday  wednesday  friday
0            Moscow   15740      11056   15944
1  Saint_Petersburg    5614       7003    5895


**Вывод:**

Данные показывают разницу поведения пользователей:

- В Москве пик прослушиваний приходится на понедельник и пятницу, а в среду заметен спад.
- В Петербурге, наоборот, больше слушают музыку по средам. Активность в понедельник и пятницу здесь почти в равной мере уступает среде.

Значит, данные говорят в пользу первой гипотезы - активность пользователей действительно зависит от дня недели, но различается в Петербурге и Москве.

### Музыка в начале и в конце недели

Согласно второй гипотезе, утром в понедельник в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.

**Сохраниv таблицы с данными в две переменные:**
* по Москве — в `moscow_general`;
* по Санкт-Петербургу — в `spb_general`.

In [31]:
moscow_general = df[df['city']=='Moscow']

In [32]:
spb_general = df[df['city']=='Saint-Petersburg']

**Создадим функцию `genre_weekday()` с четырьмя параметрами:**
* таблица (датафрейм) с данными,
* день недели,
* начальная временная метка в формате 'hh:mm',
* последняя временная метка в формате 'hh:mm'.

Функция должна вернуть информацию о топ-10 жанров тех треков, которые прослушивали в указанный день, в промежутке между двумя отметками времени.

In [33]:
def genre_weekday(df, day, time1, time2):
    # последовательная фильтрация
    # оставляем в genre_df только те строки df, у которых день равен day
    genre_df = df[df['day']==day]
    # оставляем в genre_df только те строки genre_df, у которых время меньше time2
    genre_df = genre_df[genre_df['time']<time2]
    # оставляем в genre_df только те строки genre_df, у которых время больше time1
    genre_df = genre_df[genre_df['time']>time1]
    # сгруппируем отфильтрованный датафрейм по столбцу с названиями жанров, возьмём столбец genre и посчитаем кол-во строк для каждого жанра методом count()
    genre_df_grouped = genre_df.groupby('genre')['genre'].count()
    # отсортируем результат по убыванию (чтобы в начале Series оказались самые популярные жанры)
    genre_df_sorted = genre_df_grouped.sort_values(ascending=False)
    # вернём Series с 10 самыми популярными жанрами в указанный отрезок времени заданного дня
    return genre_df_sorted[:10]

**Cравним результаты функции `genre_weekday()` для Москвы и Санкт-Петербурга в понедельник утром (с 7:00 до 11:00) и в пятницу вечером (с 17:00 до 23:00).**

**Москва, утро:**

In [34]:
genre_weekday(moscow_general, 'Monday', '07:00', '11:00')

genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
Name: genre, dtype: int64

**Петербург, утро:**

In [35]:
genre_weekday(spb_general, 'Monday', '07:00', '11:00')

genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: genre, dtype: int64

**Москва, вечер:**

In [36]:
genre_weekday(moscow_general, 'Friday', '17:00', '23:00')

genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: genre, dtype: int64

**Петербург, вечер:**

In [37]:
genre_weekday(spb_general, 'Friday', '17:00', '23:00')

genre
pop            256
electronic     216
rock           216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: genre, dtype: int64

**Выводы**

Если сравнить топ-10 жанров в понедельник утром, можно сделать такие выводы:

1. В Москве и Петербурге слушают похожую музыку. Единственное различие — в московский рейтинг вошёл жанр “world”, а в петербургский — джаз и классика.

2. В Москве пропущенных значений оказалось так много, что значение `'unknown'` заняло десятое место среди самых популярных жанров. Значит, пропущенные значения занимают существенную долю в данных и угрожают достоверности исследования.

Вечер пятницы не меняет эту картину. Некоторые жанры поднимаются немного выше, другие спускаются, но в целом топ-10 остаётся тем же самым.

Таким образом, вторая гипотеза подтвердилась лишь частично:
* Пользователи слушают похожую музыку в начале недели и в конце.
* Разница между Москвой и Петербургом не слишком выражена. В Москве чаще слушают русскую популярную музыку, в Петербурге — джаз.

Однако пропуски в данных ставят под сомнение этот результат. В Москве их так много, что рейтинг топ-10 мог бы выглядеть иначе, если бы не утерянные  данные о жанрах.

### Жанровые предпочтения в Москве и Петербурге

Гипотеза: Петербург — столица рэпа, музыку этого жанра там слушают чаще, чем в Москве.  А Москва — город контрастов, в котором, тем не менее, преобладает поп-музыка.

**Сгруппируем таблицу `moscow_general` по жанру и посчитаем прослушивания треков каждого жанра. Отсортируем результат в порядке убывания и сохраним в таблице `moscow_genres`:**

In [38]:
moscow_genres = moscow_general.groupby('genre')['genre'].count().sort_values(ascending=False)

**Выведем на экран первые десять строк `moscow_genres`:**

In [39]:
moscow_genres.head(10)

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre, dtype: int64

**Теперь повторим то же для Петербурга:**

In [40]:
spb_genres = spb_general.groupby('genre')['genre'].count().sort_values(ascending=False)
spb_genres.head(10)

genre
pop            2431
dance          1932
rock           1879
electronic     1737
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre, dtype: int64

**Выводы**

Гипотеза частично подтвердилась:
* Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.
* Вопреки ожиданиям, рэп одинаково популярен в Москве и Петербурге.


## Итоги исследования

**Мы проверили три гипотезы и установили:**

**1. День недели по-разному влияет на активность пользователей в Москве и Петербурге.**

Первая гипотеза полностью подтвердилась.

**2. Музыкальные предпочтения не сильно меняются в течение недели — будь то Москва или Петербург. Небольшие различия заметны в начале недели, по понедельникам:**
* в Москве слушают музыку жанра “world”,
* в Петербурге — джаз и классику.

Вторая гипотеза подтвердилась лишь отчасти. Этот результат мог оказаться иным, если бы не пропуски в данных.

**3. Во вкусах пользователей Москвы и Петербурга больше общего, чем различий. Вопреки ожиданиям, предпочтения жанров в Петербурге напоминают московские.**

Третья гипотеза не подтвердилась. Если различия в предпочтениях и существуют, на основной массе пользователей они незаметны.